In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor

def create_driver():
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    
    # Maximize the browser window
    driver.maximize_window()
    
    return driver

#Zoom out of the screen so that much scrolling won't be required
def set_zoom_level(driver, zoom_percentage):
    """Set the browser zoom level."""
    zoom_level = f"{zoom_percentage}%"  # e.g., '40%'
    driver.execute_script(f"document.body.style.zoom='{zoom_level}'")



def fetch_routes(driver, state_url):
    routes = []
    driver.get(state_url)
    set_zoom_level(driver, 40)
    print(f"Fetching routes from: {state_url}")

    while True:
        try:
            wait = WebDriverWait(driver, 2)
            route_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='route']")))

            for route in route_elements:
                route_name = route.text
                route_link = route.get_attribute("href")
                routes.append({"Route Name": route_name, "Route Link": route_link})

            # Handle pagination
            try:
                next_buttons = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
                active_page = driver.find_element(By.CLASS_NAME, "DC_117_pageActive")
                active_page_index = int(active_page.text)

                if active_page_index < len(next_buttons):
                    print(f"Moving to next page {active_page_index + 1}")
                    next_buttons[active_page_index].click()
                    time.sleep(1)
                else:
                    break
            except Exception as e:
                print(f"No pagination buttons found")
                break
        except Exception as e:
            print(f"Error fetching routes or no more pages: {e}")
            break
    return routes

def scroll_to_load_all_buses(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def fetch_bus_details(driver, route):
    bus_data = []
    driver.get(route["Route Link"])
    set_zoom_level(driver, 40)
    print(f"Fetching buses for route: {route['Route Name']}")

    try:
        wait = WebDriverWait(driver, 2)

        # Check and click "View Buses" button if available
        try:
            view_buses_button = wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//div[contains(@class, 'button') and contains(text(), 'View Buses')]"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_buses_button)
            time.sleep(1)
            view_buses_button.click()
            print("'View Buses' button clicked.")
        except Exception:
            print(f"No 'View Buses' button found for route: {route['Route Name']}. Proceeding with existing bus list.")

        # Scroll to load all buses
        scroll_to_load_all_buses(driver)

        # Extract all buses
        buses = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clearfix.bus-item")))

        for bus in buses:
            try:
                bus_name = bus.find_element(By.CLASS_NAME, "travels").text
                bus_type = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                price = bus.find_element(By.CLASS_NAME, "fare").text

                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text if bus.find_elements(By.CLASS_NAME, "seat-left") else "0"
                star_rating = bus.find_element(By.CLASS_NAME, "rating").find_element(By.TAG_NAME, "span").text if bus.find_elements(By.CLASS_NAME, "rating") else "0.0"

                bus_data.append({
                    "route_name": route["Route Name"],
                    "route_link": route["Route Link"],
                    "busname": bus_name,
                    "bustype": bus_type,
                    "departing_time": departing_time,
                    "duration": duration,
                    "reaching_time": reaching_time,
                    "star_rating": star_rating,
                    "price": price,
                    "seats_available": seats_available
                })
            except Exception as e:
                print(f"Error extracting bus data: {e}")

    except Exception as e:
        print(f"No buses found for route {route['Route Name']}: {e}")

    return bus_data

def process_state(state):
    driver = create_driver()
    print(f"Starting scraping for state: {state['State']}")
    state_url = state["URL"]
    bus_data = []

    try:
        routes = fetch_routes(driver, state_url)

        if routes:
            for route in routes:
                buses = fetch_bus_details(driver, route)
                print(f"Scraped {len(buses)} buses for route: {route['Route Name']}")
                bus_data.extend(buses)
        else:
            print(f"No routes found for {state['State']}.")
    except Exception as e:
        print(f"Error scraping data for state {state['State']}: {e}")
    finally:
        driver.quit()

    return bus_data

def scrape_all_states(states):
    all_bus_data = []
    max_workers = 5  # Number of parallel tabs

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_state, states))

    for result in results:
        all_bus_data.extend(result)

    # Convert to DataFrame
    df = pd.DataFrame(all_bus_data)
    print(f"Total buses scraped: {len(df)}")
    return df

# Define states
states = [
    {
        "State": "Himachal Pradesh",
        "Operator": "HRTC",
        "URL": "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"
    },
    {
        "State": "Rajasthan",
        "Operator": "RSRTC",
        "URL": "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
    },
    {
        "State": "Punjab",
        "Operator": "PEPSU",
        "URL": "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
    },
    {
        "State": "Chandigarh",
        "Operator": "CUT",
        "URL": "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
    },
    {
        "State": "Jammu & Kashmir",
        "Operator": "JKSRTC",
        "URL": "https://www.redbus.in/online-booking/jksrtc"
    },
    {
        "State": "Telangana",
        "Operator": "TGSRTC",
        "URL": "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"
    },
    {
        "State": "West Bengal",
        "Operator": "WBSTC",
        "URL": "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"
    },
    {
        "State": "Kerala",
        "Operator": "KSRTC",
        "URL": "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
    },
    {
        "State": "Bihar",
        "Operator": "BSRTC",
        "URL": "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
    },
    {
        "State": "Assam",
        "Operator": "ASTC",
        "URL": "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
    }
]

# Execute the scraper for all states
df_buses = scrape_all_states(states)
print(df_buses.head())


Starting scraping for state: RajasthanStarting scraping for state: Punjab

Starting scraping for state: Himachal Pradesh
Starting scraping for state: Chandigarh
Starting scraping for state: Jammu & Kashmir
Fetching routes from: https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu
Fetching routes from: https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile
Fetching routes from: https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile
Fetching routes from: https://www.redbus.in/online-booking/jksrtc
Fetching routes from: https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile
Moving to next page 2
Moving to next page 2
Moving to next page 2
No pagination buttons found
Moving to next page 2
Moving to next page 3
Moving to next page 3
Fetching buses for route: Delhi to Srinagar
Moving to next page 3
Fetching buses for route: Jodhpur to Ajmer
Moving to next page 4
No 'View Buses' button found for route: Delhi to Srinagar. Proceeding with 

In [4]:
df_buses

,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 2100720,Ordinary 3+2 Non AC Seater,05:00,09h 50m,14:50,4.1,INR 512,33 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,09h 25m,16:10,4.6,INR 912,26 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 592,A/C Executive (2+3),08:05,10h 05m,18:10,3.7,INR 632,37 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,09h 50m,18:40,3.7,INR 512,37 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,09h 05m,18:30,4.5,INR 912,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
2619,Ranchi to Hazaribagh,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Shivam Travels (yvts),A/C Seater (2+2),08:00,03h 00m,11:00,3.0,270,18 Seats available
2620,Ranchi to Hazaribagh,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Moti Travels,A/C Seater Push Back (2+2),17:10,02h 50m,20:00,2.8,198,39 Seats available
2621,Ranchi to Hazaribagh,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Shivam Bus Service,Mercedes Benz A/C (2+2),08:40,02h 35m,11:15,3.0,INR 300,35 Seats available
2622,Ranchi to Hazaribagh,https://www.redbus.in/bus-tickets/ranchi-to-ha...,Pammi Travels,NON A/C Seater Push Back (2+3),11:30,03h 00m,14:30,3.5,INR 200,53 Seats available


In [5]:
import mysql.connector
import pandas as pd
import re

# Function to connect to the database
def connect_to_database():
    return mysql.connector.connect(
        host="localhost",  # Update if different
        user="root",       # Update with your username
        password="root",   # Update with your password
        database="redbus", # Update with your database name
        port=3307          # Update with the correct port
    )

# Function to sanitize the DataFrame
import pandas as pd

# Function to sanitize the DataFrame
def sanitize_data(df):
    # Sanitize the 'star_rating' column (convert to float)
    df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce').fillna(0)

    # Sanitize the 'price' column (remove 'INR' and convert to numeric)
    df['price'] = df['price'].replace(r'INR ', '', regex=True)  # Remove 'INR' prefix
    df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0).astype(int)

    # Sanitize the 'seats_available' column (convert to string and extract the number of seats)
    df['seats_available'] = df['seats_available'].astype(str).str.extract('(\d+)').fillna(0).astype(int)

    #  sanitize other columns (e.g., strip spaces, handle NaNs)
    df['route_name'] = df['route_name'].str.strip().fillna('')
    df['route_link'] = df['route_link'].str.strip().fillna('')
    df['busname'] = df['busname'].str.strip().fillna('')
    df['bustype'] = df['bustype'].str.strip().fillna('')
    df['departing_time'] = df['departing_time'].str.strip().fillna('')
    df['duration'] = df['duration'].str.strip().fillna('')
    df['reaching_time'] = df['reaching_time'].str.strip().fillna('')

    return df


# Function to insert data into the database
def insert_data(df):
    connection = connect_to_database()
    cursor = connection.cursor()

    # Define the query 
    query = """
        INSERT INTO busdetails 
        (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    try:
        for index, row in df.iterrows():
            # Extract values from the DataFrame row
            values = (
                row['route_name'],
                row['route_link'],
                row['busname'],
                row['bustype'],
                row['departing_time'],
                row['duration'],
                row['reaching_time'],
                row['star_rating'],
                row['price'],
                row['seats_available']
            )
            cursor.execute(query, values)

        connection.commit()
        print("Data inserted successfully!")
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

# Sanitize the data in the DataFrame
data = df_buses
sanitized_data = sanitize_data(data)

# Insert the sanitized data
insert_data(sanitized_data)


Data inserted successfully!
